In [2]:
!python export.py -o loop

Exported: /home/ck/code/loop/dev/00a_annotations.ipynb -> loop/annotations.py
Exported: /home/ck/code/loop/dev/00b_config.ipynb -> loop/config.py
Exported: /home/ck/code/loop/dev/00c_utils.ipynb -> loop/utils.py
Exported: /home/ck/code/loop/dev/00d_mixins.ipynb -> loop/mixins.py
Exported: /home/ck/code/loop/dev/01a_callbacks.ipynb -> loop/callbacks.py
Exported: /home/ck/code/loop/dev/01b_modules.ipynb -> loop/modules.py
Exported: /home/ck/code/loop/dev/02a_metrics.ipynb -> loop/metrics.py
Exported: /home/ck/code/loop/dev/02b_phase.ipynb -> loop/phase.py
Exported: /home/ck/code/loop/dev/02c_training.ipynb -> loop/training.py
Exported: /home/ck/code/loop/dev/03a_schedule.ipynb -> loop/schedule.py
Exported: /home/ck/code/loop/dev/03b_early_stopping.ipynb -> loop/early_stopping.py
Exported: /home/ck/code/loop/dev/03c_predictions.ipynb -> loop/predictions.py
Exported: /home/ck/code/loop/dev/99_testing.ipynb -> loop/testing.py
13 notebook(s) exported into folder: loop


In [3]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [4]:
import numpy as np
from torch.utils.data import Dataset, DataLoader, TensorDataset

from loop.config import defaults
from loop.training import Loop
from loop.utils import from_torch

In [5]:
class Predictor:
    """Wraps model with """
    
    def __init__(self, model, device, batch_size: int=defaults.bs, loss_fn: 'callable'=None):
        model.to(device)
        self.model = model
        self.device = device
        self.batch_size = batch_size
        self.loss_fn = loss_fn
    
    @classmethod
    def from_loop(cls, loop: Loop, on_device=None):
        return cls(loop.model, on_device or loop.device, loop.loss_fn)
        
    def predict(self, ds: Dataset, with_gt: bool=False):
        dl = DataLoader(ds, shuffle=False, batch_size=self.batch_size)
        result, actual = [], []
        for xb, yb in dl:
            y_hat = self.model(xb.to(self.device))
            result += from_torch(y_hat).tolist()
            actual += from_torch(y_val).tolist()
        result, actual = np.array(result), np.array(actual)
        return result, actual if with_gt else result
    
    def predict_tensor(self, X, with_gt: bool=False):
        y = torch.zeros(len(X)).long()
        return self.predict(TensorDataset(X, y), with_gt)
    
    def loss(self, ds, loss_fn=None):
        loss_fn = loss_fn or self.loss_fn
        if loss_fn is None:
            raise ValueError('cannot compute loss if self.loss_fn is None')
        results, gts = self.predict(ds, with_gt=True)
        return loss_fn(results, gts)
    
    def loss_tensor(self, X, y, loss_fn=None):
        return self.loss(TensorDataset(X, y), loss_fn)

In [ ]:
class ImageClassifier(Predictor):
    
    def show_predictions(self, ds: Dataset, n: int=4, **fig_kwargs):
        results, gts = self.predict(ds, with_gt=True)
        f, axes = plt.subplots(n, n, **fig_kwargs)
        for i in range(n*n):
            ax, img, result, gt = axes.flat[i], ds[i], results[i], gts[i]
            img = from_tensor(img)
            ax.imshow(img)

In [8]:
from pathlib import Path

from torch.nn.functional import cross_entropy

from loop.callbacks import Average
from loop.early_stopping import EarlyStopping, ModelSaver
from loop.metrics import accuracy
from loop.testing import get_mnist
from loop.modules import fc_network

cbs = [
    Average(accuracy, alias='acc'), 
    EarlyStopping(metric='acc', patience=1, better=max),
    ModelSaver(mode='best', metric='acc', root=Path.home()/'models', better=max)
]

trn_ds, val_ds = get_mnist(flat=True)
loop = Loop(fc_network(784, [100, 10]), cbs=cbs, loss_fn=cross_entropy)
loop.fit_datasets(trn_ds, val_ds, epochs=10000, batch_size=100)

p = Predictor.from_loop(loop, on_device='cpu')
p.predict(val_ds)

Epoch:    1 | train_loss=0.1559, train_acc=0.9208, valid_loss=0.1259, valid_acc=0.9548


KeyboardInterrupt: 

In [ ]:
import numpy as np
from torch.nn.functional import cross_entropy
from torch.utils.data import DataLoader

from loop.callbacks import Average
from loop.modules import fc_network
from loop.metrics import accuracy
from loop.training import Loop
from loop.testing import get_mnist
from loop.utils import from_torch

cbs = [Average(accuracy, alias='acc'), 
       EarlyStopping(metric='acc', patience=1, better=max),
       ModelSaver(mode='best', metric='acc', root=Path.home()/'models', better=max)]

trn_ds, val_ds = get_mnist(flat=True)
loop = Loop(fc_network(784, [100, 10]), cbs=cbs, loss_fn=cross_entropy)
loop.fit_datasets(trn_ds, val_ds, epochs=10000, batch_size=100)

loop.cb['history'].plot()
loop.cb['model_saver'].load_last_saved_state()
loop.model.eval()

result, actual = [], []
for x_val, y_val in DataLoader(val_ds, shuffle=False, batch_size=1024):
    y_hat = loop.model(x_val.to(loop.device))
    result += from_torch(y_hat).argmax(axis=1).tolist()
    actual += from_torch(y_val).tolist()

result = np.array(result)
actual = np.array(actual)
print(f'Validation accuracy: {(result == actual).mean():2.2%}')